# Parallel reduction

<a href="https://colab.research.google.com/github/mark-hobbs/articles/blob/main/cuda/parallel-reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Literature:
- [Optimising parallel reduction in CUDA](https://developer.download.nvidia.com/assets/cuda/files/reduction.pdf)

## Reduce bond forces to particle forces

Bond forces can be stored as a bondlist or neighbour list

- bondlist [n_bonds, 2]
- neighbourlist [n_particles, n_family_members]

Reduce:
- particles.forces [n_particles, 1]

In [ ]:
import numpy as np
from numba import njit, prange

try:
    import google.colab
    !git clone https://github.com/mark-hobbs/articles.git
    import os
    os.chdir('articles/cuda')  # Navigate to the cuda subdirectory
except ImportError:
    pass  # Already local, no need to clone

import utils

Cloning into 'articles'...
remote: Enumerating objects: 453, done.
remote: Counting objects: 100% (311/311), done.
remote: Compressing objects: 100% (244/244), done.
remote: Total 453 (delta 138), reused 230 (delta 60), pack-reused 142 (from 1)
Receiving objects: 100% (453/453), 97.54 MiB | 15.23 MiB/s, done.
Resolving deltas: 100% (200/200), done.


In [ ]:
np.random.seed(42)
n_particles = 1500000
n_family_members = 128

bond_forces = np.random.rand(n_particles, n_family_members)

### Numpy and Numba

In [ ]:
@utils.profile(runs=10)
def reduce_bond_forces_a(bond_forces):
    n_particles = bond_forces.shape[0]
    f = np.zeros((n_particles))
    for i in range(n_particles):
        f[i] = np.sum(bond_forces[i, :])
    return f

In [ ]:
@utils.profile(runs=10)
def reduce_bond_forces_b(bond_forces):
    return np.sum(bond_forces, axis=1)

In [ ]:
@utils.profile(runs=10)
@njit(parallel=True, fastmath=True)
def reduce_bond_forces_c(bond_forces):
    n_particles = bond_forces.shape[0]
    f = np.zeros((n_particles))
    for i in prange(n_particles):
        f[i] = np.sum(bond_forces[i, :])
    return f

In [ ]:
f_a = reduce_bond_forces_a(bond_forces)
f_b = reduce_bond_forces_b(bond_forces)
f_c = reduce_bond_forces_c(bond_forces)
assert np.allclose(f_a, f_b) and np.allclose(f_b, f_c), "Results are not equal"

Function 'reduce_bond_forces_a' executed 10 time(s)
Average execution time: 5.2300 seconds
Min: 4.8264s, Max: 5.8252s

Function 'reduce_bond_forces_b' executed 10 time(s)
Average execution time: 0.1300 seconds
Min: 0.1277s, Max: 0.1356s

Function 'reduce_bond_forces_c' executed 10 time(s)
Average execution time: 0.2869 seconds
Min: 0.1124s, Max: 1.8379s



### Numba CUDA

In [ ]:
from numba import cuda, float32

In [ ]:
@cuda.jit
def row_sum_kernel(neighbourlist, output):
    row = cuda.blockIdx.x
    tid = cuda.threadIdx.x
    n_cols = neighbourlist.shape[1]

    # Allocate shared memory for each thread to load one value
    sdata = cuda.shared.array(256, dtype=float32)  # Adjust size if needed

    val = 0.0
    if tid < n_cols:
        val = neighbourlist[row, tid]

    sdata[tid] = val
    cuda.syncthreads()

    s = cuda.blockDim.x // 2
    while s > 0:
        if tid < s and tid + s < n_cols:
            sdata[tid] += sdata[tid + s]
        cuda.syncthreads()
        s //= 2

    if tid == 0:
        output[row] = sdata[0]

@utils.profile(runs=10)
def reduce_bond_forces_gpu(neighbourlist):
    n_particles, n_family_members = neighbourlist.shape
    threads_per_block = 256  # Match shared memory allocation
    shared_mem = threads_per_block * 4  # float32: 4 bytes

    d_neigh = cuda.to_device(neighbourlist.astype(np.float32))
    d_out = cuda.device_array(n_particles, dtype=np.float32)

    row_sum_kernel[n_particles, threads_per_block](d_neigh, d_out)
    return d_out.copy_to_host()

In [ ]:
utils.get_cuda_device_info()

CUDA Device Information:
----------------------------------------
CUDA Runtime Version:          12.5
Device Name:                   b'Tesla T4'
Compute Capability:            (7, 5)

Memory:
Total Memory:                  15.83 GB
Free Memory:                   15.72 GB

Compute Resources:
Streaming Multiprocessors:     40
Max Threads per Block:         1024

Grid Limitations:
Max Grid Dimensions X:         2147483647
Max Grid Dimensions Y:         65535
Max Grid Dimensions Z:         65535

Additional Characteristics:
Warp Size:                     32
Clock Rate:                    1.59 GHz
Memory Clock Rate:             5.00 GHz


See [this post](https://github.com/googlecolab/colabtools/issues/5081) to understand compatability issues with Google Colab and Numba CUDA

In [ ]:
!uv pip install -q --system numba-cuda==0.4.0

In [ ]:
from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

In [ ]:
f_gpu = reduce_bond_forces_gpu(bond_forces)
assert np.allclose(f_a, f_gpu), "Results are not equal"

Function 'reduce_bond_forces_gpu' executed 10 time(s)
Average execution time: 0.5765 seconds
Min: 0.4644s, Max: 1.2543s



In [ ]:
import time

def benchmark_kernel(neighbourlist, num_runs=100):
    n_particles, n_family_members = neighbourlist.shape
    threads_per_block = 256

    d_neigh = cuda.to_device(neighbourlist.astype(np.float32))
    d_out = cuda.device_array(n_particles, dtype=np.float32)

    # Warm up the kernel
    for _ in range(5):
        row_sum_kernel[n_particles, threads_per_block](d_neigh, d_out)

    cuda.synchronize()

    start_time = time.perf_counter()

    for _ in range(num_runs):
        row_sum_kernel[n_particles, threads_per_block](d_neigh, d_out)

    cuda.synchronize()

    end_time = time.perf_counter()

    avg_time = (end_time - start_time) / num_runs
    return avg_time, d_out.copy_to_host()

In [ ]:
cuda_event_time, result = benchmark_kernel(bond_forces, num_runs=100)
print(f"Kernel executed in {cuda_event_time:.4f} seconds")

Kernel executed in 0.0172 seconds
